# Jazykové slovníky

In [ ]:
# Vynucení kontroly souladu s PEP8
!pip install flake8 pycodestyle pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

Slovníky programu Aspell (a dalších jemu podobných) se skládají ze dvou částí – ze seznamu základů slov (`*.wl`) a ze seznamu předpon/přípon (`*_affix.dat`). Pravidla pro jejich skládání jsou následující:

- Slovo ve slovníku `*.wl` se vyskytuje na vlastní řádce buď celé nebo doplněné příznaky afixu /ZNAKY.
- Uvedené ZNAKY jsou odkazy do odpovídajícího souboru `*_affix.wl` na stejně pojmenované skupiny (co znak, to skupina), které určují, jakým dalším způsobem může být uvedený slovníkový záznam upraven.
- První řádek každé skupiny identifikuje tuto skupinu, určuje, zda je možno míchat předpony s příponami nebo ne (Y/N) a udává počet jejích záznamů (tedy následujících řádek se stejnou identifikací).
- Každý následující řádek afixové skupiny pak postupně udává:
    1. příznak předpony/přípony (aneb prefixu/sufixu);
    1. identifikátor afixové skupiny (to jsou ta písmenka za lomítkem);
    1. znaky, které je třeba vypustit z konce uvedeného slova, než může být přidána sekvence znaků z následujícího sloupce (0 udává, že se žádný znak nevypouští);
    1. sekvence znaků pro přidání k základu slova (0 udává, že se žádná sekvence nepřidává);
    1. (regexpová) podmínka pro konec základu slova (tedy toho tvaru před lomítkem ve slovníku &ast;.wl), která musí být splněna, aby se vůbec uvažovala pravidla ze třetího a čtvrtého sloupce.
    
(Blíže viz Affix Compression nebo spíš rovnou příslušnou manuálovou stránku Hunspellu.)


### 1. Vaším úkolem (nepřekvapivě :-) je z uvedených souborů vyrobit maximální možné slovníky pro český a slovenský jazyk.

**Note:** Kódování jednotlivých souborů je popsáno buď přímo v nich (pro případ affixů) nebo v přiloženém souboru `*.dat`. 
Zdroj: Aspell.net

**Hint:** Slovníky jsou veliké a byť zpracování je nakonec celkem jednoduché, tak se asi vyplatí odladit funkcionalitu tvoření slov na menší podmnožině slov a teprve poté ji pustit na celý slovník. Nezapomeňte na možnost kombinace prefixů a sufixů, u jazyků jako čeština to nepřekvapivě dělá divy s výsledným počtem.
PS: Já se u češtiny na druhý pokus dopočítal k číslu 4 269 031. Děkuji studentům předmětu BI-PYT v zimním semestru 2018/2019, kteří mě upozornili, že při prvním pokusu jsem zapomněl na kombinaci prefixů a sufixů dohromady a ještě ke všemu i odstranění duplikátů z nagenerovaných stejných tvarů ^_^"

In [ ]:
cs_dat = 'aspell_cs/cs_affix.dat'
cs_wl = 'aspell_cs/cs.wl'

In [ ]:
from pprint import pprint
import re
from itertools import product


#
# pomocné funkce
#
def read_block(data):
    blok = []
    for d in data:
        if d == '':
            yield blok
            blok = []
        else:
            blok.append(d)


def add_affix(zaklad, typ, affix):
    # někdy není co přidat..
    if affix == '0':
        return zaklad
    # ..ale většinou ano
    if typ == 'PFX':
        return affix + zaklad
    elif typ == 'SFX':
        return zaklad + affix


def add_affixes(zaklad, prefix, sufix):
    # někdy není co přidat..
    if prefix == '0' and sufix == '0':
        print('AFFIXy 0 pro', zaklad)
        return zaklad
    # ..ale většinou ano
    return (
        (prefix if prefix != '0' else '')
        + zaklad
        + (sufix if sufix != '0' else '')
    )


def make_words(zaklad, affixy):
    slova = []
    cross = {'PFX': '', 'SFX': ''}
    for affix in affixy:
        try:
            pravidla = afixes[affix]
        except:
            print('AFFIXY:', affixy, '@', zaklad)
            exit('AFFIX error')
        for pravidlo in pravidla:
            try:
                typ, _, pryc, retezec, podminka, *_ = pravidlo
            except:
                print('PRAVIDLO:', pravidlo)
                exit('PRAVIDLO error')
            # ověření podmínky
            pattern = re.compile(podminka + r'$')
            matches = pattern.search(zaklad)
            if not matches:
                continue
            # aplikace affixů
            if pryc == '0':
                t = add_affix(zaklad, typ, retezec)
            else:
                t = add_affix(zaklad[: -(len(pryc))], typ, retezec)
            slova.append(t)
        # Je affix kombinovatelný?
        if affix in combinable:
            cross[typ] += affix
    if cross['PFX'] and cross['SFX']:  # != ''
        for pfx, sfx in product(cross['PFX'], cross['SFX']):
            pravidla_pfx, pravidla_sfx = afixes[pfx], afixes[sfx]
            for p, s in product(pravidla_pfx, pravidla_sfx):
                pfx_typ, _, pfx_pryc, pfx_retezec, pfx_podminka, *_ = p
                sfx_typ, _, sfx_pryc, sfx_retezec, sfx_podminka, *_ = s
                # ověření podmínek
                pfx_pattern = re.compile(pfx_podminka + r'$')
                pfx_matches = pfx_pattern.search(zaklad)
                sfx_pattern = re.compile(sfx_podminka + r'$')
                sfx_matches = sfx_pattern.search(zaklad)
                if not (pfx_matches and sfx_matches):
                    continue
                # aplikace affixů
                # TODO: pro zjednodušení zatím předpokládám, že prefixy nechtějí nic od základu odtrhávat (což je pravda u "cs_affix.dat", ale jinde by nemusela)
                if sfx_pryc == '0':
                    t = add_affixes(zaklad, pfx_retezec, sfx_retezec)
                else:
                    t = add_affixes(
                        zaklad[: -(len(sfx_pryc))], pfx_retezec, sfx_retezec
                    )
                slova.append(t)
    return slova


#
# čeština
#

# affixy
with open(cs_dat, encoding='iso8859-2') as f:
    affix = f.read().split('\n')[3:]
afixes = {}
combinable = ''
for blok in read_block(affix):
    if blok:
        head, *tail = blok
        _, ida, comb, _ = head.split()
        if comb == 'Y':
            combinable += ida
        afixes[ida] = [tuple(' '.join(t.split()).split()) for t in tail]

# slova
# with open('cs/TEST.cs.wl', encoding='iso8859-2') as f:
with open(cs_wl, encoding='iso8859-2') as f:
    data = f.read().split('\n')
print('Základů slov:', len(data))  # 306217

with open('cestina.txt', 'w', encoding='utf-8') as f:
    for zaznam in data:
        if '/' in zaznam:
            t = zaznam.split('/')
            f.write(t[0] + '\n')
            for slovo in make_words(*t):
                f.write(slovo + '\n')
        else:
            f.write(zaznam + '\n')

"""
Výsledkem je 4 686 986 slov (resp. jejich tvarů).
"""

### 2. Pokračujmež v předchozím příkladu: Když už máte maximální možné slovníky pro češtinu a slovenštinu, proveďte nad nimi následující analýzu:

- Kolik slov obsahují tyto vytvořené slovníky?
- Jaká část slovní zásoby obou jazyků je úplně stejná?
- Jaké je rozdělení slov obou jazyků podle jejich délky?

**Hint:** Možná se vám bude hodit, že Python obsahuje datový typ množina.

In [ ]:
from collections import defaultdict

# 1
with open('slovencina.txt', mode='r', encoding='utf-8') as f:
    data_sk = set(f.read().strip().split('\n'))
    print('Slovenských slov:', len(data_sk))  # 1003961

with open('cestina.txt', mode='r', encoding='utf-8') as f:
    data_cs = set(f.read().strip().split('\n'))
    print('Českých slov:', len(data_cs))  # 3047386

# 2
společná = data_cs & data_sk
print('Společný průnik:', len(společná))  # 167296

# 3
analyza_sk = defaultdict(int)
for slovo in data_sk:
    analyza_sk[len(slovo)] += 1
    # if len(slovo) == 44: print(slovo)
with open('analyza_sk.log', mode='w', encoding='ascii') as f:
    for k, v in analyza_sk.items():
        f.write(str(k) + '\t' + str(v) + '\n')

analyza_cs = defaultdict(int)
for slovo in data_cs:
    analyza_cs[len(slovo)] += 1
with open('analyza_cs.log', mode='w', encoding='ascii') as f:
    for k, v in analyza_cs.items():
        f.write(str(k) + '\t' + str(v) + '\n')

"""
Slovenských slov: 1532767
Českých slov: 4269031
Společný průnik: 238733

PS -- 44 znaků dlouhá slova ve slovenštině:
    dvadsaťdvatisícdvestosedemdesiattrinásobnými
    dvadsaťdvatisícdvestosedemdesiattrinásobných
"""

### 3. A ještě jedna varianta na předchozí zadání: Použijte údaje o distribuci délky slov v obou jazycích a vyneste ji do nějakého pěkného grafu.

In [ ]:
import matplotlib.pyplot as plt

# data...
data_sk = []
with open('analyza_sk.log', mode='r', encoding='ascii') as f:
    for line in f:
        data_sk.append(line.strip().split('\t'))
data_cs = []
with open('analyza_cs.log', mode='r', encoding='ascii') as f:
    for line in f:
        data_cs.append(line.strip().split('\t'))

# ...a jejich histogramy
# a) CS
xd, yd = [], []
for d in data_cs:
    xd.append(int(d[0]))
    yd.append(int(d[1]))
plt.subplot(2, 1, 1)
plt.xlim(0, 45)
plt.bar(xd, yd, color='g')
plt.ylabel('čeština')
plt.title('Četnosti délky slov v češtině a slovenštině')
# b) SK
xd, yd = [], []
for d in data_sk:
    xd.append(int(d[0]))
    yd.append(int(d[1]))
plt.subplot(2, 1, 2)
plt.xlim(0, 45)
plt.bar(xd, yd, color='y')
plt.xlabel('počet písmen ve slově')
plt.ylabel('slovenština')

# zobrazení grafu
plt.show()